In [2]:
%matplotlib inline
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

import os
import cv2
import numpy as np
import torch
from torch.autograd import Variable
import quat_math
import pickle

from PIL import Image
import scipy.io as scio
from functools import partial
from object_pose_utils.utils import to_np, to_var

import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = 20, 12
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


In [3]:
from quat_math import quat2AxisAngle

def quats2Point(quats):
    pts = []
    for q in quats:
        xi, theta = quat2AxisAngle(q)
        pts.append(xi*theta)
    return np.array(pts)

def scatterSO3(vertices, vals, q_gt = None, alpha_max = 1.0):
    #c = cmap(vals)
    a = np.maximum(0,np.minimum(1, (vals - min(vals))/(max(vals) - min(vals))))
    c = cmap(a)
    
    c[:,3] = alpha_max*a

    pts = quats2Point(vertices)
    ax.scatter(pts[:,0], pts[:,1], pts[:,2], s=a*10, c=c)

    if(q_gt is not None):
        pt_gt = quats2Point(q_gt)
        ax.scatter(pt_gt[:,0], pt_gt[:,1], pt_gt[:,2], c='r', marker='x')

In [4]:
from transforms3d.quaternions import quat2mat, mat2quat

def getPoseCNNQuat(data, obj):
    pose_idx = np.where(data['rois'][:,1].flatten()==obj)[0]
    if(len(pose_idx) == 0):
        return None
    else:
        pose_idx = pose_idx[0]
    pose = data['poses'][pose_idx]
    q = pose[:4][[1,2,3,0]]
    q /= np.linalg.norm(q)
    t = pose[4:7]
    return q


In [5]:
from object_pose_utils.datasets.ycb_dataset import YcbDataset as YCBDataset
from object_pose_utils.datasets.feature_dataset import FeatureDataset

from object_pose_utils.datasets.image_processing import ImageNormalizer
from object_pose_utils.datasets.pose_dataset import OutputTypes as otypes

dataset_root = '/ssd0/datasets/ycb/YCB_Video_Dataset'
object_list = list(range(1,22))
mode = "test"

output_format = [otypes.OBJECT_LABEL,
                 otypes.QUATERNION, 
                 otypes.IMAGE_CROPPED, 
                 otypes.DEPTH_POINTS_MASKED_AND_INDEXES]



In [26]:
test_feature_root = '/scratch/bokorn/results/dense_fusion_local_feat_test/'
feature_root = '/scratch/bokorn/results/dense_fusion_local_orig_feat/'
feature_key = 'feat_global'

feature_size = 1024

sigmas_data = np.load('/home/bokorn/src/DenseFusion/df_orig_sigmas.npz', allow_pickle=True)
sigmas_single = sigmas_data['single_sigma'].item()
sigmas_mixture = sigmas_data['mixture_sigma'].item()

from se3_distributions.models.compare_networks import SigmoidCompareNet

estimator_comp = SigmoidCompareNet(feature_size, 21)
estimator_comp.load_state_dict(torch.load(comp_model_checkpoint))
estimator_comp.cuda();

from object_pose_utils.utils.interpolation import BinghamInterpolation, TetraInterpolation
from object_pose_utils.utils.bingham import isobingham_likelihood
tetra_interp = TetraInterpolation(2)

In [9]:
conf = np.load('conf_mat_valid_syn_grid_offset/{}_confusion_matrix.npy'.format(1))
conf[0].sum()

0.0

In [40]:
isobingham_likelihood(df_quat.cuda(), 
                                torch.tensor(sigmas_single[obj.item()]).unsqueeze(0).float().cuda(),
                                grid_vertices.cuda()).shape

torch.Size([3885, 1])

In [60]:
from se3_distributions.datasets.ycb_dataset import getYCBSymmeties
from object_pose_utils.utils.pose_processing import symmetricAngularDistance, meanShift, unique_tol, tensorAngularDiff
import pathlib


import scipy.io as scio
from tqdm import tqdm_notebook as tqdm

grid_size = 3885

lik = {}
err = {}

with torch.no_grad():
    for object_id in tqdm(object_list):
        sym_axis, sym_ang = getYCBSymmeties(object_id)
        ycb_dataset = FeatureDataset(dataset_root = dataset_root,
                                     feature_root = test_feature_root,
                                     feature_key = feature_key,
                                     resample_on_error = False,
                                     mode=mode, object_list = [object_id])
        grid_vertices = torch.load(os.path.join(feature_root, 'grid',
            '{}_vertices.pt'.format(ycb_dataset.classes[object_id])))
        grid_size = grid_vertices.shape[0]
        
        confusion_matrix = np.load('conf_mat_valid_syn_grid_offset/{}_confusion_matrix.npy'.format(object_id))
        
        lik[object_id] = []
        err[object_id] = []
        
        pathlib.Path('results/data'.format(ycb_dataset.classes[object_id])).mkdir(parents=True, exist_ok=True)
        
        for j, data in tqdm(enumerate(ycb_dataset), total=len(ycb_dataset)):
            obj, feat, quat = data
            if(len(obj) == 0):
                lik[object_id].append(np.nan)
                err[object_id].append(np.nan)
                continue
            
            feature_data = np.load('{}/data/{}_{}_feat.npz'.format(ycb_dataset.feature_root, 
                ycb_dataset.getPath(j), ycb_dataset.classes[obj]))
            df_quat = torch.tensor(feature_data['max_q']).float()

            max_idx = torch.argmin(tensorAngularDiff(df_quat, grid_vertices))
            dist_est = confusion_matrix[max_idx] + 0.000001
            
            gt_idx = torch.argmin(tensorAngularDiff(df_quat, grid_vertices))

            tetra_interp.setValues(dist_est.flatten())
            #lik[object_id].append(tetra_interp.smooth(to_np(quat)).item())
            lik[object_id].append(dist_est[gt_idx]/ sum(dist_est))
            
            mode_quat = grid_vertices[np.argmax(dist_est)].unsqueeze(0)
            
            err[object_id].append(symmetricAngularDistance(mode_quat, quat.unsqueeze(0),
                                             sym_axis, sym_ang).item()*180/np.pi)

            savepath = 'results/{}'.format(ycb_dataset.getPath(j))
            pathlib.Path(savepath).mkdir(parents=True, exist_ok=True)

            if(False):
                scio.savemat('{}/{}_{}_sigma_results_ROT.mat'.format(savepath, 
                                                    ycb_dataset.classes[object_id], 
                                                    ycb_dataset.feature_key),
                        {'index':j,
                         'quat':to_np(quat),
                         'quat_est':to_np(df_quat),
                         'mode_quat':to_np(mode_quat),
                         'dist_est':dist_est,
                         'err':err[object_id][-1],
                         'lik':lik[object_id][-1],
                         })
#np.savez('results/local_global_conf_ROT.npz', 
#         lik=lik,
#         err=err,)

Exception on index 49: [Errno 2] No such file or directory: '/scratch/bokorn/results/dense_fusion_local_feat_test/data/0050/000633_005_tomato_soup_can_feat.npz'
Exception on index 50: [Errno 2] No such file or directory: '/scratch/bokorn/results/dense_fusion_local_feat_test/data/0050/000640_005_tomato_soup_can_feat.npz'
Exception on index 51: [Errno 2] No such file or directory: '/scratch/bokorn/results/dense_fusion_local_feat_test/data/0050/000642_005_tomato_soup_can_feat.npz'
Exception on index 52: [Errno 2] No such file or directory: '/scratch/bokorn/results/dense_fusion_local_feat_test/data/0050/000644_005_tomato_soup_can_feat.npz'
Exception on index 54: [Errno 2] No such file or directory: '/scratch/bokorn/results/dense_fusion_local_feat_test/data/0050/000653_005_tomato_soup_can_feat.npz'
Exception on index 55: [Errno 2] No such file or directory: '/scratch/bokorn/results/dense_fusion_local_feat_test/data/0050/000660_005_tomato_soup_can_feat.npz'
Exception on index 57: [Errno 2] N

Exception on index 87: [Errno 2] No such file or directory: '/scratch/bokorn/results/dense_fusion_local_feat_test/data/0049/000833_010_potted_meat_can_feat.npz'
Exception on index 108: [Errno 2] No such file or directory: '/scratch/bokorn/results/dense_fusion_local_feat_test/data/0049/001157_010_potted_meat_can_feat.npz'
Exception on index 111: [Errno 2] No such file or directory: '/scratch/bokorn/results/dense_fusion_local_feat_test/data/0049/001179_010_potted_meat_can_feat.npz'
Exception on index 115: [Errno 2] No such file or directory: '/scratch/bokorn/results/dense_fusion_local_feat_test/data/0049/001222_010_potted_meat_can_feat.npz'
Exception on index 116: [Errno 2] No such file or directory: '/scratch/bokorn/results/dense_fusion_local_feat_test/data/0049/001229_010_potted_meat_can_feat.npz'
Exception on index 118: [Errno 2] No such file or directory: '/scratch/bokorn/results/dense_fusion_local_feat_test/data/0049/001247_010_potted_meat_can_feat.npz'
Exception on index 129: [Errn

In [63]:
max(lik[5])/(np.pi**2/3885)

273.82795535025895

In [33]:
np.savez('results/local_global_max_conf.npz', 
         lik=lik,
         err=err,)

2373.0

{1: [1669.000001,
  1669.000001,
  1669.000001,
  1669.000001,
  1669.000001,
  1669.000001,
  1669.000001,
  1669.000001,
  1669.000001,
  457.000001,
  457.000001,
  1669.000001,
  1669.000001,
  457.000001,
  457.000001,
  457.000001,
  1669.000001,
  1669.000001,
  1669.000001,
  1669.000001,
  1669.000001,
  1669.000001,
  1669.000001,
  457.000001,
  457.000001,
  960.000001,
  457.000001,
  457.000001,
  457.000001,
  457.000001,
  457.000001,
  457.000001,
  457.000001,
  457.000001,
  457.000001,
  457.000001,
  457.000001,
  457.000001,
  457.000001,
  457.000001,
  457.000001,
  457.000001,
  457.000001,
  457.000001,
  457.000001,
  457.000001,
  457.000001,
  457.000001,
  457.000001,
  457.000001,
  457.000001,
  457.000001,
  457.000001,
  457.000001,
  457.000001,
  457.000001,
  457.000001,
  457.000001,
  457.000001,
  457.000001,
  457.000001,
  457.000001,
  457.000001,
  457.000001,
  457.000001,
  457.000001,
  457.000001,
  457.000001,
  727.000001,
  727.000001,